In [11]:
from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
import pickle 
import re 
from collections import defaultdict
import datetime
import codecs
import json


In [12]:
# This method is done for you. Make sure to put your credentials in twitter.cfg.
def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [13]:
def robust_request(twitter, resource, params, max_tries=2):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 401:
            print 'Unauthorized data access. The user does not have access to such information'
            break
        elif "page does not exist" in request.text:
            print 'page does not exist'
            break
        elif request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)
 

In [14]:
def get_followers_from_pickle(filename):
    """ Returns all followers of a Twitter account by retrieving saved data,
        using pickle method.
    
    Args: A string representing the screen_name of a Twitter account,
          whose followers we need.
    Returns : A dictionary of { screen_name : user_info } pairs of followers,
              where user_info is a dictionary object containing user information
    """
    fileObject = open(filename,'r')
    followers = pickle.load(fileObject)
    fileObject.close()
    return followers

# fname = 'streaming_user_tweets_2015-11-18 12:26:03.txt'
# data = get_followers_from_pickle(fname)
# print data
followers_list = get_followers_from_pickle('nu_followers.txt')
len(followers)

35984

In [13]:
# def get_followers():
#     followers = []
#     f = open ('iit_followers.txt','rb')
#     regexp = re.compile(r'p\d+')
#     for line in f:
#         if regexp.search(str(line)) is not None:
#             continue
#         else: 
#             if line.startswith('aV'):
#                 line = line[2:]
#                 followers.append(line.replace("\n",""))
#     followers.sort()
#     return followers

# followers_list = get_followers()

In [15]:
#This method collects tweets of all the followers of IIT's different twitter profiles and pickles them to a file
#Each file is time stamped and creates a new file at every 5000 tweets
#It takes the name of the output file and index in followers_list (list of screen_names of iit followers) 
#where one would like to start data collection. 
def print_tweets(outputfile,follower_index):
    tweets = 0
    followers = followers_list[follower_index:]
    f = codecs.open(outputfile,'a',encoding='utf8')
    for follower in followers:
        print "processing follower %s with index %d in followers_list" %(follower, followers_list.index(follower))
        request = robust_request(twitter,'statuses/user_timeline',{'screen_name': follower})
        if request is not None:
            if tweets <= 10000:
                for tweet in request:
                    if tweet is not None:
                        tweets += 1
                        json.dump(tweet,f)
                        f.write('\n')
                        print 'dumped tweet no %d' %(tweets)
            else:
                timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
                newfile = 'nu_outputfile_' + timestamp + '.txt'
                print 'calling new file name', newfile
                print_tweets(newfile,followers_list.index(follower))
    print 'End of collecting tweets of followers'
        

outputfile = 'nu_outputfile_'+ datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+ '.txt'
print_tweets(outputfile,0)


Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


processing follower 005ia with index 0 in followers_list
dumped tweet no 1
dumped tweet no 2
dumped tweet no 3
dumped tweet no 4
dumped tweet no 5
dumped tweet no 6
dumped tweet no 7
dumped tweet no 8
dumped tweet no 9
dumped tweet no 10
dumped tweet no 11
processing follower 007MAV with index 1 in followers_list
dumped tweet no 12
dumped tweet no 13
dumped tweet no 14
dumped tweet no 15
dumped tweet no 16
dumped tweet no 17
dumped tweet no 18
dumped tweet no 19
dumped tweet no 20
dumped tweet no 21
dumped tweet no 22
dumped tweet no 23
dumped tweet no 24
dumped tweet no 25
dumped tweet no 26
dumped tweet no 27
dumped tweet no 28
dumped tweet no 29
dumped tweet no 30
dumped tweet no 31
processing follower 007kengom with index 2 in followers_list
dumped tweet no 32
dumped tweet no 33
dumped tweet no 34
dumped tweet no 35
dumped tweet no 36
dumped tweet no 37
dumped tweet no 38
dumped tweet no 39
dumped tweet no 40
dumped tweet no 41
dumped tweet no 42
dumped tweet no 43
dumped tweet no 

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 2017
dumped tweet no 2018
dumped tweet no 2019
dumped tweet no 2020
dumped tweet no 2021
dumped tweet no 2022
dumped tweet no 2023
dumped tweet no 2024
dumped tweet no 2025
dumped tweet no 2026
dumped tweet no 2027
dumped tweet no 2028
dumped tweet no 2029
dumped tweet no 2030
dumped tweet no 2031
dumped tweet no 2032
dumped tweet no 2033
dumped tweet no 2034
dumped tweet no 2035
processing follower 2854music with index 180 in followers_list
dumped tweet no 2036
dumped tweet no 2037
dumped tweet no 2038
dumped tweet no 2039
dumped tweet no 2040
dumped tweet no 2041
dumped tweet no 2042
dumped tweet no 2043
dumped tweet no 2044
dumped tweet no 2045
dumped tweet no 2046
dumped tweet no 2047
dumped tweet no 2048
dumped tweet no 2049
dumped tweet no 2050
dumped tweet no 2051
dumped tweet no 2052
dumped tweet no 2053
dumped tweet no 2054
dumped tweet no 2055
processing follower 28LCW with index 181 in followers_list
Unauthorized data access. The user does not have access to

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 4378
dumped tweet no 4379
dumped tweet no 4380
dumped tweet no 4381
dumped tweet no 4382
dumped tweet no 4383
dumped tweet no 4384
dumped tweet no 4385
dumped tweet no 4386
dumped tweet no 4387
dumped tweet no 4388
dumped tweet no 4389
dumped tweet no 4390
dumped tweet no 4391
dumped tweet no 4392
dumped tweet no 4393
dumped tweet no 4394
processing follower ABRAHAMLABRADA with index 360 in followers_list
dumped tweet no 4395
dumped tweet no 4396
dumped tweet no 4397
dumped tweet no 4398
dumped tweet no 4399
dumped tweet no 4400
dumped tweet no 4401
dumped tweet no 4402
dumped tweet no 4403
dumped tweet no 4404
dumped tweet no 4405
dumped tweet no 4406
dumped tweet no 4407
dumped tweet no 4408
dumped tweet no 4409
dumped tweet no 4410
dumped tweet no 4411
dumped tweet no 4412
dumped tweet no 4413
dumped tweet no 4414
processing follower ABSALLBZ with index 361 in followers_list
dumped tweet no 4415
dumped tweet no 4416
dumped tweet no 4417
dumped tweet no 4418
dumped t


dumped tweet no 5712
dumped tweet no 5713
dumped tweet no 5714
dumped tweet no 5715
dumped tweet no 5716
dumped tweet no 5717
dumped tweet no 5718
dumped tweet no 5719
dumped tweet no 5720
dumped tweet no 5721
dumped tweet no 5722
dumped tweet no 5723
dumped tweet no 5724
dumped tweet no 5725
dumped tweet no 5726
dumped tweet no 5727
dumped tweet no 5728
dumped tweet no 5729
dumped tweet no 5730
processing follower AIMBiomedical with index 441 in followers_list


TwitterConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=5)

In [ ]:
# followers_list.index('AIMBiomedical')
outputfile = 'nu_outputfile_'+ datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+ '.txt'
print_tweets(outputfile,441)


processing follower AIMBiomedical with index 441 in followers_list
dumped tweet no 1
dumped tweet no 2
dumped tweet no 3
dumped tweet no 4
dumped tweet no 5
dumped tweet no 6
dumped tweet no 7
dumped tweet no 8
dumped tweet no 9
dumped tweet no 10
dumped tweet no 11
dumped tweet no 12
dumped tweet no 13
dumped tweet no 14
dumped tweet no 15
dumped tweet no 16
dumped tweet no 17
dumped tweet no 18
dumped tweet no 19
dumped tweet no 20
processing follower AIM_Companies with index 442 in followers_list
dumped tweet no 21
dumped tweet no 22
dumped tweet no 23
dumped tweet no 24
dumped tweet no 25
dumped tweet no 26
dumped tweet no 27
dumped tweet no 28
dumped tweet no 29
dumped tweet no 30
dumped tweet no 31
dumped tweet no 32
dumped tweet no 33
dumped tweet no 34
dumped tweet no 35
dumped tweet no 36
dumped tweet no 37
dumped tweet no 38
dumped tweet no 39
dumped tweet no 40
processing follower AIPCareers with index 443 in followers_list
dumped tweet no 41
dumped tweet no 42
dumped tweet 